# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 00 Game Info

In [ ]:
STATE_SIZE = (3,3) #틱택토 보드 크기
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)

# 01 HYPER PARAMS

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# 02 Env+State

In [ ]:
import os

# 파일 복사 명령어 실행
os.system('cp "/content/drive/My Drive/Colab Notebooks/공통 environment+state.ipynb" "/content/"')

0

In [ ]:
import nbformat

notebook_path = "/content/공통 environment+state.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    notebook_content = nbformat.read(f, as_version=4)

# 각 코드 셀 출력 및 실행
for cell in notebook_content.cells:
    if cell.cell_type == "code":
        print(f"실행 중인 코드:\n{cell.source}\n{'='*40}")
        exec(cell.source)

실행 중인 코드:
import torch
import torch.nn as nn
import numpy as np
실행 중인 코드:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)
실행 중인 코드:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
실행 중인 코드:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.n ** 2
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win': 1, 'lose': -1, 'draw': 0} #결과에 따른 보상 정의
        #승리 1점 / 패배 -1점 / 무승부 0점


    def step(self, present_state, action_idx): #현재 상태에서 주어진 행동에 따라 게임 진행 / 행동 결과를 계산
        """
        present_state에 대해 action_idx의 행동에 따라 게임을 한 턴 진행시키고
        next_state, is_done, is_lose를 반환한다.
        """
        #action_idx: 플레이어가 선택한 행동의 인덱스 / 보드 칸
        #현재 상태에서 행동하는 행동을 수행하여 다음 상태를 계산
        next_state = present_state.next(action_idx)
        is_done, is_lose = ne

# 03 MCS

In [ ]:
class MCSAgent: # 몬테카를로 탐색 알고리즘 클래스
    def __init__(self, n_simulations=1000):
        self.n_simulations = n_simulations

    # 시뮬레이션 실행
    # 주어진 상태와 행동에서 시뮬레이션을 실행하고 결과를 반환
    def run_simulation(self, state, action):
        current_state = state.next(action)  # 행동 수행 후의 상태

        while True:
            is_done, _ = current_state.check_done()
            if is_done:
                break  # 게임이 종료되면 루프 탈출

            # 가능한 행동 중 무작위로 선택하여 진행
            legal_actions = np.where(current_state.get_legal_actions() != 0)[0]
            if len(legal_actions) == 0:
                break  # 더 이상 가능한 행동이 없으면 종료

            # 무작위 행동 선택 (순수한 MCS)
            action = np.random.choice(legal_actions)
            current_state = current_state.next(action)

        # 게임이 종료되면 보상을 반환
        return current_state.get_reward(current_state)


    # 각 행동에 대한 시뮬레이션 수행
    def evaluate_actions(self, state):
        legal_actions = np.where(state.get_legal_actions() != 0)[0]
        action_scores = {action: [] for action in legal_actions}

        for action in legal_actions:  # 각 가능한 행동에 대해 시뮬레이션 수행
            for _ in range(self.n_simulations):
                result = self.run_simulation(state, action)
                if result is not None:
                    action_scores[action].append(result)

        # 각 행동에 대한 평균 보상 계산 / 값이 없을 경우 기본값 0 설정
        action_means = {
            action: np.mean(scores) if len(scores) > 0 else 0
            for action, scores in action_scores.items()
        }
        return action_means


    def get_action(self, state): #시뮬레이션 결과를 기반으로 최적의 행동 반환
        action_means = self.evaluate_actions(state)

        # 예외 처리: 가능한 행동이 없는 경우 none 반환 방지
        if not action_means:
           legal_actions = np.where(state.get_legal_actions() != 0)[0]
           if len(legal_actions) > 0:
               print("[WARNING] No optimal actions found, choosing a random action.")
               return np.random.choice(legal_actions)  # 가능한 행동 중 랜덤 선택
           else:
               print("[ERROR] No valid actions available. Game might be in an invalid state.")
               return None  # 안전하게 None 반환


        # 평균 점수가 가장 높은 행동을 찾음 / 동점이면 랜덤
        max_value = max(action_means.values())
        best_actions = [action for action, value in action_means.items() if value == max_value]  # 동점일 경우 랜덤 선택
        return np.random.choice(best_actions)

## Test Code

In [ ]:
# 1. State 초기화
state = State()
print("초기 상태 보드:")
state.render(state)
print("가능한 행동들:", np.where(state.get_legal_actions() != 0)[0])

초기 상태 보드:
가능한 행동들: [0 1 2 3 4 5 6 7 8]


In [ ]:
# 2. MonteCarloSearch 초기화
monte_carlo = MCSAgent(n_simulations=1000)

In [ ]:
# 3. 행동 평가 및 최적 행동 확인
action_means = monte_carlo.evaluate_actions(state)
print("각 행동의 평균 점수:", action_means)

best_action = monte_carlo.get_action(state)
print("몬테카를로 탐색이 선택한 최적 행동:", best_action)

각 행동의 평균 점수: {0: -0.746, 1: -0.776, 2: -0.772, 3: -0.754, 4: -0.755, 5: -0.756, 6: -0.759, 7: -0.759, 8: -0.76}
몬테카를로 탐색이 선택한 최적 행동: 7
